# Statistical Process Control (SPC)

## Purpose

This notebook teaches you how to apply Statistical Process Control (SPC) methods to monitor and control manufacturing processes in AM-QADF. You'll learn SPC fundamentals, control chart generation, process capability analysis, multivariate SPC, control rule detection, and baseline calculation using a unified interactive interface with real-time progress tracking and detailed logging.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Understand SPC concepts and their importance in quality control
- ✅ Generate various control charts (X-bar, R, S, Individual, Moving Range)
- ✅ Analyze process capability using indices (Cp, Cpk, Pp, Ppk)
- ✅ Perform multivariate SPC analysis (Hotelling T², PCA-based)
- ✅ Detect control rule violations (Western Electric, Nelson rules)
- ✅ Establish and update process baselines with adaptive limits
- ✅ Monitor quality metrics using SPC control charts
- ✅ Generate comprehensive SPC reports with visualizations
- ✅ Interpret SPC results and make process improvement decisions
- ✅ Monitor SPC analysis progress with real-time status and logs

## Estimated Duration

90-120 minutes

---

## Overview

Statistical Process Control (SPC) is a method of quality control that uses statistical methods to monitor and control manufacturing processes. The AM-QADF SPC module provides:

- 📊 **Control Charts**: X-bar, R, S, Individual, and Moving Range charts
- 📈 **Process Capability**: Cp, Cpk, Pp, Ppk indices and capability analysis
- 🔬 **Multivariate SPC**: Hotelling T² and PCA-based multivariate monitoring
- 🎯 **Control Rules**: Western Electric and Nelson rule detection
- 📐 **Baseline Calculation**: Establish and update process baselines
- 🔄 **Adaptive Limits**: Dynamically update control limits based on new data
- 📊 **Real-Time Monitoring**: Track progress with status bars and detailed execution logs
- ⏱️ **Time Tracking**: Monitor execution time for all SPC operations

The notebook features a unified interactive interface with:
- **Progress Tracking**: Visual progress bars showing completion percentage
- **Status Monitoring**: Real-time status updates with elapsed time
- **Detailed Logging**: Timestamped logs with success/warning/error indicators for all operations
- **Error Handling**: Comprehensive error messages and tracebacks in the logs

Use the interactive widgets below to perform SPC analysis - no coding required! Monitor your SPC analysis progress in real-time using the status bar and logs section at the bottom.

## Interactive Statistical Process Control Interface

Use the widgets below to perform SPC analysis, generate control charts, analyze process capability, detect rule violations, and establish baselines. All SPC tasks are organized systematically in one unified interface!

In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple,
    HTML as WidgetHTML, Textarea, FileUpload
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import json
from typing import Optional, Tuple, Dict, Any, List

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import SPC classes
SPC_AVAILABLE = False
spc_client = None

try:
    from am_qadf.analytics.spc import (
        SPCClient, SPCConfig,
        ControlChartGenerator, ControlChartResult,
        ProcessCapabilityAnalyzer, ProcessCapabilityResult,
        MultivariateSPCAnalyzer, MultivariateSPCResult,
        ControlRuleDetector, ControlRuleViolation,
        BaselineCalculator, BaselineStatistics, AdaptiveLimitsCalculator
    )
    SPC_AVAILABLE = True
    print("✅ SPC classes available")
except ImportError as e:
    print(f"⚠️ SPC classes not available: {e} - using demo mode")

# Try to import quality assessment client with SPC
quality_client = None
try:
    from am_qadf.analytics.quality_assessment.client import QualityAssessmentClient
    quality_client = QualityAssessmentClient(enable_spc=SPC_AVAILABLE, mongo_client=None)
    print("✅ Quality assessment client with SPC available")
except ImportError as e:
    print(f"⚠️ Quality assessment client not available: {e}")

# MongoDB connection setup (optional, for loading real data)
INFRASTRUCTURE_AVAILABLE = False
mongo_client = None
voxel_storage = None

try:
    from src.infrastructure.config import MongoDBConfig
    from src.infrastructure.database import MongoDBClient
    from am_qadf.voxel_domain import VoxelGridStorage
    
    config = MongoDBConfig.from_env()
    if not config.username:
        config.username = os.getenv('MONGO_ROOT_USERNAME', 'admin')
    if not config.password:
        config.password = os.getenv('MONGO_ROOT_PASSWORD', 'password')
    
    mongo_client = MongoDBClient(config=config)
    # Check connection with error handling
    try:
        if mongo_client.is_connected():
            voxel_storage = VoxelGridStorage(mongo_client=mongo_client)
            INFRASTRUCTURE_AVAILABLE = True
            print(f"✅ Connected to MongoDB: {config.database}")
        else:
            mongo_client = None
            print("⚠️ MongoDB connection failed - using demo mode")
    except Exception as conn_error:
        mongo_client = None
        print(f"⚠️ MongoDB connection check failed: {conn_error} - using demo mode")
except ImportError as e:
    print(f"⚠️ MongoDB infrastructure not available: {e} - using demo mode")
    mongo_client = None
except Exception as e:
    print(f"⚠️ MongoDB not available: {e} - using demo mode")
    mongo_client = None

print("✅ Setup complete!")

✅ Environment variables loaded from development.env
✅ SPC classes available
✅ Quality assessment client with SPC available
✅ Connected to MongoDB: am_qadf_data
✅ Setup complete!


In [2]:
# Create Interactive SPC Interface

# Global state
spc_results = {}
control_chart_results = {}
capability_results = {}
multivariate_results = {}
rule_violation_results = {}
baseline_results = {}
spc_reports = {}
current_spc_type = None
operation_start_time = None

# Initialize SPC client if available
if SPC_AVAILABLE:
    spc_config = SPCConfig(
        control_limit_sigma=3.0,
        subgroup_size=5,
        baseline_sample_size=100,
        adaptive_limits=False
    )
    spc_client = SPCClient(config=spc_config, mongo_client=mongo_client if INFRASTRUCTURE_AVAILABLE else None)
    print("✅ SPC client initialized")
else:
    spc_client = None
    print("⚠️ SPC client not available - using demo mode")

# ============================================
# Helper Functions for Demo Data Generation
# ============================================

def generate_demo_process_data(n_samples=100, mean=10.0, std=1.0, subgroup_size=5, include_outliers=False):
    """Generate demo process data for control charts."""
    np.random.seed(42)
    if subgroup_size > 1:
        n_subgroups = n_samples // subgroup_size
        data = []
        for i in range(n_subgroups):
            subgroup = np.random.normal(mean, std, subgroup_size)
            if include_outliers and i % 10 == 0:
                subgroup[0] += 4 * std
            data.append(subgroup)
        return np.array(data)
    else:
        data = np.random.normal(mean, std, n_samples)
        if include_outliers:
            data[::10] += 4 * std
        return data

def generate_demo_multivariate_data(n_samples=100, n_variables=3, correlation=0.7):
    """Generate demo multivariate process data."""
    np.random.seed(42)
    cov_matrix = np.eye(n_variables) * 0.5
    for i in range(n_variables):
        for j in range(i+1, n_variables):
            cov_matrix[i, j] = correlation
            cov_matrix[j, i] = correlation
    mean = np.zeros(n_variables)
    return np.random.multivariate_normal(mean, cov_matrix, n_samples)

def generate_demo_capability_data(n_samples=200, mean=10.0, std=1.0):
    """Generate demo data for capability analysis."""
    np.random.seed(42)
    return np.random.normal(mean, std, n_samples)

# ============================================
# Top Panel: SPC Type Selection and Actions
# ============================================

spc_type_label = WidgetHTML("<b>SPC Analysis Type:</b>")
spc_type = RadioButtons(
    options=[
        ('Control Charts', 'control_charts'),
        ('Process Capability', 'capability'),
        ('Multivariate SPC', 'multivariate'),
        ('Control Rules', 'rules'),
        ('Baseline Calculation', 'baseline'),
        ('Comprehensive Analysis', 'comprehensive')
    ],
    value='control_charts',
    description='Type:',
    style={'description_width': 'initial'}
)

data_source_label = WidgetHTML("<b>Data Source:</b>")
data_source_mode = RadioButtons(
    options=[('Demo Data', 'demo'), ('MongoDB', 'mongodb')],
    value='demo',
    description='Source:',
    style={'description_width': 'initial'}
)

execute_button = Button(
    description='Execute SPC Analysis',
    button_style='success',
    icon='check',
    layout=Layout(width='200px')
)

export_button = Button(
    description='Export Report',
    button_style='',
    icon='download',
    layout=Layout(width='150px')
)

top_panel = VBox([
    HBox([spc_type_label, spc_type]),
    HBox([data_source_label, data_source_mode, execute_button, export_button])
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Configuration Accordion
# ============================================

# 1. Control Charts Configuration
control_charts_label = WidgetHTML("<b>Control Chart Configuration:</b>")
chart_type = Dropdown(
    options=[
        ('X-bar', 'xbar'),
        ('R Chart', 'r'),
        ('S Chart', 's'),
        ('Individual', 'individual'),
        ('Moving Range', 'moving_range'),
        ('X-bar & R', 'xbar_r'),
        ('X-bar & S', 'xbar_s')
    ],
    value='xbar',
    description='Chart Type:',
    style={'description_width': 'initial'}
)

subgroup_size = IntSlider(
    value=5,
    min=2,
    max=20,
    step=1,
    description='Subgroup Size:',
    style={'description_width': 'initial'}
)

control_limit_sigma = FloatSlider(
    value=3.0,
    min=2.0,
    max=4.0,
    step=0.1,
    description='Control Limit (σ):',
    style={'description_width': 'initial'}
)

enable_warnings = Checkbox(
    value=True,
    description='Enable Warning Limits (2σ)'
)

n_samples = IntSlider(
    value=100,
    min=20,
    max=500,
    step=10,
    description='Sample Size:',
    style={'description_width': 'initial'}
)

control_charts_config = VBox([
    control_charts_label,
    chart_type,
    subgroup_size,
    control_limit_sigma,
    enable_warnings,
    n_samples
], layout=Layout(padding='5px', border='1px solid #ddd'))

# 2. Process Capability Configuration
capability_label = WidgetHTML("<b>Process Capability Configuration:</b>")
spec_usl = FloatText(
    value=14.0,
    description='USL:',
    style={'description_width': 'initial'}
)

spec_lsl = FloatText(
    value=6.0,
    description='LSL:',
    style={'description_width': 'initial'}
)

target_value = FloatText(
    value=10.0,
    description='Target:',
    style={'description_width': 'initial'}
)

capability_sample_size = IntSlider(
    value=200,
    min=30,
    max=1000,
    step=10,
    description='Sample Size:',
    style={'description_width': 'initial'}
)

capability_config = VBox([
    capability_label,
    spec_usl,
    spec_lsl,
    target_value,
    capability_sample_size
], layout=Layout(padding='5px', border='1px solid #ddd'))

# 3. Multivariate SPC Configuration
multivariate_label = WidgetHTML("<b>Multivariate SPC Configuration:</b>")
multivariate_method = RadioButtons(
    options=[('Hotelling T²', 'hotelling_t2'), ('PCA-based', 'pca')],
    value='hotelling_t2',
    description='Method:',
    style={'description_width': 'initial'}
)

n_variables = IntSlider(
    value=3,
    min=2,
    max=10,
    step=1,
    description='# Variables:',
    style={'description_width': 'initial'}
)

multivariate_sample_size = IntSlider(
    value=100,
    min=30,
    max=500,
    step=10,
    description='Sample Size:',
    style={'description_width': 'initial'}
)

n_components = IntSlider(
    value=2,
    min=1,
    max=5,
    step=1,
    description='PCA Components:',
    style={'description_width': 'initial'}
)

multivariate_config = VBox([
    multivariate_label,
    multivariate_method,
    n_variables,
    multivariate_sample_size,
    n_components
], layout=Layout(padding='5px', border='1px solid #ddd'))

# 4. Control Rules Configuration
rules_label = WidgetHTML("<b>Control Rules Configuration:</b>")
rule_set = RadioButtons(
    options=[('Western Electric', 'western_electric'), ('Nelson', 'nelson'), ('Both', 'both')],
    value='both',
    description='Rule Set:',
    style={'description_width': 'initial'}
)

enable_all_rules = Checkbox(
    value=True,
    description='Enable All Rules'
)

rules_config = VBox([
    rules_label,
    rule_set,
    enable_all_rules
], layout=Layout(padding='5px', border='1px solid #ddd'))

# 5. Baseline Configuration
baseline_label = WidgetHTML("<b>Baseline Configuration:</b>")
baseline_sample_size = IntSlider(
    value=100,
    min=30,
    max=500,
    step=10,
    description='Baseline Samples:',
    style={'description_width': 'initial'}
)

adaptive_limits = Checkbox(
    value=False,
    description='Enable Adaptive Limits'
)

update_frequency = IntSlider(
    value=50,
    min=10,
    max=200,
    step=10,
    description='Update Frequency:',
    style={'description_width': 'initial'}
)

baseline_method = RadioButtons(
    options=[('Exponential Smoothing', 'exponential_smoothing'), ('Cumulative', 'cumulative')],
    value='exponential_smoothing',
    description='Update Method:',
    style={'description_width': 'initial'}
)

baseline_config = VBox([
    baseline_label,
    baseline_sample_size,
    adaptive_limits,
    update_frequency,
    baseline_method
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Combine into Accordion
config_accordion = Accordion(children=[
    control_charts_config,
    capability_config,
    multivariate_config,
    rules_config,
    baseline_config
])

config_accordion.set_title(0, '📊 Control Charts')
config_accordion.set_title(1, '📈 Process Capability')
config_accordion.set_title(2, '🔬 Multivariate SPC')
config_accordion.set_title(3, '🎯 Control Rules')
config_accordion.set_title(4, '📐 Baseline')

left_panel = VBox([
    WidgetHTML("<h3>SPC Configuration</h3>"),
    config_accordion
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Visualization and Results
# ============================================

viz_mode = RadioButtons(
    options=[
        ('Control Charts', 'charts'),
        ('Process Capability', 'capability'),
        ('Multivariate SPC', 'multivariate'),
        ('Rule Violations', 'rules'),
        ('Baseline Statistics', 'baseline'),
        ('Comprehensive Report', 'report')
    ],
    value='charts',
    description='View:',
    style={'description_width': 'initial'}
)

main_output = Output(layout=Layout(height='600px', overflow='auto'))

center_panel = VBox([
    WidgetHTML("<h3>SPC Results</h3>"),
    viz_mode,
    main_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Status and Summary
# ============================================

status_label = WidgetHTML("<b>Status:</b>")
status_display = WidgetHTML("Ready for SPC analysis")

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)

status_section = VBox([
    status_label,
    status_display,
    progress_bar
], layout=Layout(padding='5px', border='2px solid #4CAF50'))

results_summary_label = WidgetHTML("<b>Results Summary:</b>")
results_summary_display = WidgetHTML("No SPC analysis executed yet")
results_summary_section = VBox([
    results_summary_label,
    results_summary_display
], layout=Layout(padding='5px'))

metrics_display_label = WidgetHTML("<b>Key Metrics:</b>")
metrics_display = WidgetHTML("No metrics available")
metrics_section = VBox([
    metrics_display_label,
    metrics_display
], layout=Layout(padding='5px'))

spc_status_label = WidgetHTML("<b>SPC Status:</b>")
spc_status_display = WidgetHTML("Not analyzed")
spc_status_section = VBox([
    spc_status_label,
    spc_status_display
], layout=Layout(padding='5px'))

# SPC logs output
spc_logs = Output(layout=Layout(height='200px', border='1px solid #ccc', overflow_y='auto'))

# Initialize logs
with spc_logs:
    display(HTML("<p><i>SPC logs will appear here...</i></p>"))

# Bottom status bar
bottom_status = WidgetHTML(value='<b>Status:</b> Ready | <b>Progress:</b> 0% | <b>Time:</b> 0:00')
bottom_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Overall:',
    bar_style='info',
    layout=Layout(width='100%')
)

warning_display = WidgetHTML("")

right_panel = VBox([
    status_section,
    results_summary_section,
    metrics_section,
    spc_status_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# Enhanced bottom panel
bottom_panel = VBox([
    WidgetHTML("<b>Status:</b>"),
    status_display,
    WidgetHTML("<b>SPC Analysis Logs:</b>"),
    spc_logs,
    WidgetHTML("<hr>"),
    bottom_status,
    bottom_progress,
    warning_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Logging Functions
# ============================================

def log_message(message: str, level: str = 'info'):
    """Log a message to the SPC logs with timestamp and emoji."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    icons = {'info': 'ℹ️', 'success': '✅', 'warning': '⚠️', 'error': '❌'}
    icon = icons.get(level, 'ℹ️')
    with spc_logs:
        print(f"[{timestamp}] {icon} {message}")

def update_status(operation: str, progress: int = None):
    """Update the status display and progress."""
    global operation_start_time
    status_display.value = f"<b>{operation}</b>"
    if progress is not None:
        progress_bar.value = progress
        bottom_progress.value = progress
        if operation_start_time:
            elapsed = time.time() - operation_start_time
            mins, secs = divmod(int(elapsed), 60)
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> {mins}:{secs:02d}'
        else:
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> 0:00'
    else:
        if operation_start_time:
            elapsed = time.time() - operation_start_time
            mins, secs = divmod(int(elapsed), 60)
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress_bar.value}% | <b>Time:</b> {mins}:{secs:02d}'
        else:
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress_bar.value}% | <b>Time:</b> 0:00'

print("✅ SPC interface initialized!")

✅ SPC client initialized
✅ SPC interface initialized!


In [3]:
# ============================================
# Execution Functions for Each SPC Analysis Type
# ============================================

def execute_spc_analysis(b):
    """Execute SPC analysis based on selected type."""
    global operation_start_time
    operation_start_time = time.time()
    
    # Clear logs
    with spc_logs:
        clear_output(wait=True)
        display(HTML("<p><i>SPC logs will appear here...</i></p>"))
    
    with main_output:
        clear_output(wait=True)
        analysis_type = spc_type.value
        
        log_message(f"Starting {spc_type.label}...", 'info')
        update_status(f"Executing {spc_type.label}...", 0)
        
        try:
            if analysis_type == 'control_charts':
                execute_control_charts()
            elif analysis_type == 'capability':
                execute_process_capability()
            elif analysis_type == 'multivariate':
                execute_multivariate_spc()
            elif analysis_type == 'rules':
                execute_control_rules()
            elif analysis_type == 'baseline':
                execute_baseline_calculation()
            elif analysis_type == 'comprehensive':
                execute_comprehensive_workflow()
            
            log_message(f"{spc_type.label} completed successfully", 'success')
            update_status(f"{spc_type.label} complete", 100)
        except Exception as e:
            log_message(f"Error during {spc_type.label}: {str(e)}", 'error')
            import traceback
            log_message(f"Traceback: {traceback.format_exc()}", 'error')
            warning_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
            update_status(f"Error during {spc_type.label}", 0)

def execute_control_charts():
    """Execute control chart generation."""
    log_message("Control Chart Generation", 'info')
    log_message(f"Chart Type: {chart_type.label}", 'info')
    log_message(f"Subgroup Size: {subgroup_size.value}, Sample Size: {n_samples.value}", 'info')
    update_status("Generating demo data...", 10)
    
    # Generate demo data
    data = generate_demo_process_data(
        n_samples=n_samples.value,
        subgroup_size=subgroup_size.value if chart_type.value in ['xbar', 'r', 's', 'xbar_r', 'xbar_s'] else 1,
        include_outliers=False
    )
    
    log_message(f"Generated {len(data) if isinstance(data, np.ndarray) and data.ndim == 1 else data.shape[0]} samples", 'info')
    
    print("📊 Control Chart Generation")
    print("=" * 60)
    print(f"Chart Type: {chart_type.label}")
    print(f"Subgroup Size: {subgroup_size.value}")
    print(f"Control Limit: {control_limit_sigma.value}σ")
    print(f"Warning Limits: {'Enabled' if enable_warnings.value else 'Disabled'}")
    print()
    
    if SPC_AVAILABLE and spc_client:
        try:
            update_status("Creating control chart...", 30)
            log_message("Creating control chart with SPC client...", 'info')
            
            # Update config
            spc_client.config.control_limit_sigma = control_limit_sigma.value
            spc_client.config.subgroup_size = subgroup_size.value
            spc_client.config.enable_warnings = enable_warnings.value
            
            # Create chart based on type
            if chart_type.value == 'xbar_r':
                result = spc_client.create_control_chart(data, chart_type='xbar_r')
            elif chart_type.value == 'xbar_s':
                result = spc_client.create_control_chart(data, chart_type='xbar_s')
            else:
                result = spc_client.create_control_chart(data, chart_type=chart_type.value)
            
            control_chart_results['latest'] = result
            
            update_status("Chart generated successfully", 80)
            log_message(f"Control limits: UCL={result.upper_control_limit:.3f}, LCL={result.lower_control_limit:.3f}", 'success')
            log_message(f"Out-of-control points: {len(result.out_of_control_points)}", 'info' if len(result.out_of_control_points) == 0 else 'warning')
            
            print("✅ Control Chart Generated!")
            print(f"\n📊 Results:")
            print(f"   Center Line: {result.center_line:.3f}")
            print(f"   UCL: {result.upper_control_limit:.3f}")
            print(f"   LCL: {result.lower_control_limit:.3f}")
            print(f"   Out-of-Control Points: {len(result.out_of_control_points)}")
            
            # Visualization
            fig, ax = plt.subplots(figsize=(12, 6))
            sample_indices = result.sample_indices if result.sample_indices is not None else np.arange(len(result.sample_values))
            ax.plot(sample_indices, result.sample_values, 'b-o', markersize=4, label='Sample Values', alpha=0.7)
            ax.axhline(result.center_line, color='g', linestyle='-', linewidth=2, label='Center Line')
            ax.axhline(result.upper_control_limit, color='r', linestyle='--', linewidth=2, label='UCL')
            ax.axhline(result.lower_control_limit, color='r', linestyle='--', linewidth=2, label='LCL')
            
            if result.upper_warning_limit is not None and enable_warnings.value:
                ax.axhline(result.upper_warning_limit, color='orange', linestyle=':', linewidth=1, label='UWL (2σ)')
                ax.axhline(result.lower_warning_limit, color='orange', linestyle=':', linewidth=1, label='LWL (2σ)')
            
            # Highlight out-of-control points
            if len(result.out_of_control_points) > 0:
                ooc_indices = [sample_indices[i] for i in result.out_of_control_points]
                ooc_values = [result.sample_values[i] for i in result.out_of_control_points]
                ax.scatter(ooc_indices, ooc_values, color='red', s=100, marker='x', linewidths=3, label='Out-of-Control', zorder=5)
            
            ax.set_xlabel('Sample Number')
            ax.set_ylabel('Sample Value')
            ax.set_title(f'{chart_type.label} Control Chart')
            ax.legend()
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            status_display.value = f"✅ Control chart generated: {len(result.out_of_control_points)} OOC points"
            results_summary_display.value = f"Chart: {chart_type.label}<br>Center: {result.center_line:.3f}<br>UCL: {result.upper_control_limit:.3f}<br>LCL: {result.lower_control_limit:.3f}<br>OOC: {len(result.out_of_control_points)}"
            metrics_display.value = f"Center Line: {result.center_line:.3f}<br>UCL: {result.upper_control_limit:.3f}<br>LCL: {result.lower_control_limit:.3f}<br>Range: {result.upper_control_limit - result.lower_control_limit:.3f}"
            spc_status_display.value = "✓ In Control" if len(result.out_of_control_points) == 0 else "⚠️ Out of Control"
            
        except Exception as e:
            log_message(f"Control chart generation failed: {str(e)}", 'error')
            print(f"❌ Control chart generation failed: {e}")
            status_display.value = f"❌ Error: {e}"
            warning_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
    else:
        # Demo mode - create simple visualization
        log_message("Running control chart in demo mode...", 'warning')
        update_status("Creating demo chart...", 50)
        
        center_line = np.mean(data) if data.ndim == 1 else np.mean([np.mean(sub) for sub in data])
        std_dev = np.std(data) if data.ndim == 1 else np.std([np.mean(sub) for sub in data])
        
        ucl = center_line + control_limit_sigma.value * std_dev
        lcl = center_line - control_limit_sigma.value * std_dev
        uwl = center_line + 2.0 * std_dev if enable_warnings.value else None
        lwl = center_line - 2.0 * std_dev if enable_warnings.value else None
        
        sample_values = data if data.ndim == 1 else np.array([np.mean(sub) for sub in data])
        sample_indices = np.arange(len(sample_values))
        ooc_points = [i for i, val in enumerate(sample_values) if val > ucl or val < lcl]
        
        log_message(f"Demo chart - Center: {center_line:.3f}, UCL: {ucl:.3f}, LCL: {lcl:.3f}", 'info')
        log_message(f"Out-of-control points: {len(ooc_points)}", 'warning' if len(ooc_points) > 0 else 'info')
        
        print("✅ Control Chart Generated! (Demo Mode)")
        print(f"\n📊 Results (Demo):")
        print(f"   Center Line: {center_line:.3f}")
        print(f"   UCL: {ucl:.3f}")
        print(f"   LCL: {lcl:.3f}")
        print(f"   Out-of-Control Points: {len(ooc_points)}")
        
        # Visualization
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(sample_indices, sample_values, 'b-o', markersize=4, label='Sample Values', alpha=0.7)
        ax.axhline(center_line, color='g', linestyle='-', linewidth=2, label='Center Line')
        ax.axhline(ucl, color='r', linestyle='--', linewidth=2, label='UCL')
        ax.axhline(lcl, color='r', linestyle='--', linewidth=2, label='LCL')
        
        if uwl is not None and lwl is not None:
            ax.axhline(uwl, color='orange', linestyle=':', linewidth=1, label='UWL (2σ)')
            ax.axhline(lwl, color='orange', linestyle=':', linewidth=1, label='LWL (2σ)')
        
        if len(ooc_points) > 0:
            ax.scatter([sample_indices[i] for i in ooc_points], [sample_values[i] for i in ooc_points],
                      color='red', s=100, marker='x', linewidths=3, label='Out-of-Control', zorder=5)
        
        ax.set_xlabel('Sample Number')
        ax.set_ylabel('Sample Value')
        ax.set_title(f'{chart_type.label} Control Chart (Demo Mode)')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        update_status("Control chart complete (demo)", 80)
        status_display.value = f"✅ Control chart complete (demo): {len(ooc_points)} OOC points"
        results_summary_display.value = f"Chart: {chart_type.label} (Demo)<br>Center: {center_line:.3f}<br>UCL: {ucl:.3f}<br>LCL: {lcl:.3f}<br>OOC: {len(ooc_points)}"
        metrics_display.value = f"Center Line: {center_line:.3f}<br>UCL: {ucl:.3f}<br>LCL: {lcl:.3f}<br>Range: {ucl - lcl:.3f}"
        spc_status_display.value = "✓ In Control" if len(ooc_points) == 0 else "⚠️ Out of Control"

def execute_process_capability():
    """Execute process capability analysis."""
    log_message("Process Capability Analysis", 'info')
    update_status("Generating demo data...", 10)
    
    data = generate_demo_capability_data(n_samples=capability_sample_size.value)
    usl_val = spec_usl.value
    lsl_val = spec_lsl.value
    target_val = target_value.value
    
    log_message(f"Spec limits: LSL={lsl_val}, USL={usl_val}, Target={target_val}", 'info')
    
    print("📈 Process Capability Analysis")
    print("=" * 60)
    print(f"LSL: {lsl_val}, USL: {usl_val}, Target: {target_val}")
    print()
    
    if SPC_AVAILABLE and spc_client:
        try:
            update_status("Calculating capability indices...", 30)
            result = spc_client.analyze_process_capability(data, specification_limits=(lsl_val, usl_val), target_value=target_val)
            capability_results['latest'] = result
            
            update_status("Capability analysis complete", 80)
            log_message(f"Cp: {result.cp:.3f}, Cpk: {result.cpk:.3f}", 'success')
            log_message(f"Pp: {result.pp:.3f}, Ppk: {result.ppk:.3f}", 'info')
            log_message(f"Capability Rating: {result.capability_rating}", 'info')
            
            print("✅ Capability Analysis Complete!")
            print(f"\n📊 Capability Indices:")
            print(f"   Cp: {result.cp:.3f}")
            print(f"   Cpk: {result.cpk:.3f}")
            print(f"   Pp: {result.pp:.3f}")
            print(f"   Ppk: {result.ppk:.3f}")
            print(f"   Rating: {result.capability_rating}")
            
            status_display.value = f"✅ Capability: Cpk={result.cpk:.3f} ({result.capability_rating})"
            metrics_display.value = f"Cp: {result.cp:.3f}<br>Cpk: {result.cpk:.3f}<br>Pp: {result.pp:.3f}<br>Ppk: {result.ppk:.3f}"
            spc_status_display.value = result.capability_rating
            
        except Exception as e:
            log_message(f"Capability analysis failed: {str(e)}", 'error')
            print(f"❌ Capability analysis failed: {e}")
            status_display.value = f"❌ Error: {e}"
    else:
        # Demo mode
        log_message("Running capability analysis in demo mode...", 'warning')
        mean = np.mean(data)
        std = np.std(data, ddof=1)
        cp = (usl_val - lsl_val) / (6 * std) if std > 0 else 0
        cpu = (usl_val - mean) / (3 * std) if std > 0 else 0
        cpl = (mean - lsl_val) / (3 * std) if std > 0 else 0
        cpk = min(cpu, cpl)
        
        print("✅ Capability Analysis Complete! (Demo Mode)")
        print(f"\n📊 Capability Indices (Demo):")
        print(f"   Cp: {cp:.3f}")
        print(f"   Cpk: {cpk:.3f}")
        
        status_display.value = f"✅ Capability complete (demo): Cpk={cpk:.3f}"
        metrics_display.value = f"Cp: {cp:.3f}<br>Cpk: {cpk:.3f}"
        update_status("Capability analysis complete (demo)", 80)

def execute_multivariate_spc():
    """Execute multivariate SPC analysis."""
    log_message("Multivariate SPC Analysis", 'info')
    log_message(f"Method: {multivariate_method.label}", 'info')
    update_status("Generating demo data...", 10)
    
    data = generate_demo_multivariate_data(n_samples=multivariate_sample_size.value, n_variables=n_variables.value)
    
    log_message(f"Generated {data.shape[0]} samples with {data.shape[1]} variables", 'info')
    
    print("🔬 Multivariate SPC Analysis")
    print("=" * 60)
    print(f"Method: {multivariate_method.label}")
    print(f"Variables: {n_variables.value}, Samples: {multivariate_sample_size.value}")
    print()
    
    if SPC_AVAILABLE and spc_client:
        try:
            update_status("Creating multivariate chart...", 30)
            result = spc_client.create_multivariate_chart(data, method=multivariate_method.value)
            multivariate_results['latest'] = result
            
            update_status("Multivariate analysis complete", 80)
            log_message(f"Generated {multivariate_method.label} chart", 'success')
            
            print("✅ Multivariate SPC Complete!")
            print(f"\n📊 Results:")
            print(f"   Method: {multivariate_method.label}")
            print(f"   Samples: {data.shape[0]}")
            print(f"   Variables: {data.shape[1]}")
            
            status_display.value = f"✅ Multivariate SPC: {multivariate_method.label}"
            results_summary_display.value = f"Method: {multivariate_method.label}<br>Samples: {data.shape[0]}<br>Variables: {data.shape[1]}"
            
        except Exception as e:
            log_message(f"Multivariate SPC failed: {str(e)}", 'error')
            print(f"❌ Multivariate SPC failed: {e}")
            status_display.value = f"❌ Error: {e}"
    else:
        log_message("Running multivariate SPC in demo mode...", 'warning')
        print("✅ Multivariate SPC Complete! (Demo Mode)")
        update_status("Multivariate SPC complete (demo)", 80)
        status_display.value = f"✅ Multivariate SPC complete (demo): {multivariate_method.label}"

def execute_control_rules():
    """Execute control rule detection."""
    log_message("Control Rule Detection", 'info')
    log_message(f"Rule Set: {rule_set.label}", 'info')
    update_status("Generating demo chart data...", 10)
    
    # First create a control chart
    data = generate_demo_process_data(n_samples=100, include_outliers=True)
    
    if SPC_AVAILABLE and spc_client:
        try:
            update_status("Creating control chart...", 30)
            chart_result = spc_client.create_control_chart(data, chart_type='xbar')
            
            update_status("Detecting rule violations...", 50)
            if rule_set.value == 'western_electric':
                violations = spc_client.detect_rule_violations(chart_result, rules='western_electric')
            elif rule_set.value == 'nelson':
                violations = spc_client.detect_rule_violations(chart_result, rules='nelson')
            else:
                violations = spc_client.detect_rule_violations(chart_result, rules='both')
            
            rule_violation_results['latest'] = violations
            
            update_status("Rule detection complete", 80)
            log_message(f"Detected {len(violations)} rule violations", 'warning' if len(violations) > 0 else 'success')
            
            print("✅ Control Rule Detection Complete!")
            print(f"\n📊 Rule Violations:")
            print(f"   Total Violations: {len(violations)}")
            for i, violation in enumerate(violations[:5], 1):
                print(f"   {i}. {violation.rule_name}: {violation.description} (Severity: {violation.severity})")
            if len(violations) > 5:
                print(f"   ... and {len(violations) - 5} more violations")
            
            status_display.value = f"✅ Rule detection: {len(violations)} violations found"
            results_summary_display.value = f"Rule Set: {rule_set.label}<br>Violations: {len(violations)}"
            
        except Exception as e:
            log_message(f"Rule detection failed: {str(e)}", 'error')
            print(f"❌ Rule detection failed: {e}")
            status_display.value = f"❌ Error: {e}"
    else:
        log_message("Running rule detection in demo mode...", 'warning')
        print("✅ Control Rule Detection Complete! (Demo Mode)")
        update_status("Rule detection complete (demo)", 80)
        status_display.value = "✅ Rule detection complete (demo)"

def execute_baseline_calculation():
    """Execute baseline calculation."""
    log_message("Baseline Calculation", 'info')
    update_status("Generating baseline data...", 10)
    
    data = generate_demo_process_data(n_samples=baseline_sample_size.value)
    
    log_message(f"Generating baseline from {len(data)} samples", 'info')
    
    print("📐 Baseline Calculation")
    print("=" * 60)
    print(f"Sample Size: {baseline_sample_size.value}")
    print()
    
    if SPC_AVAILABLE and spc_client:
        try:
            update_status("Calculating baseline...", 30)
            baseline = spc_client.establish_baseline(data, subgroup_size=subgroup_size.value)
            baseline_results['latest'] = baseline
            
            update_status("Baseline calculation complete", 80)
            log_message(f"Baseline mean: {baseline.mean:.3f}, std: {baseline.std:.3f}", 'success')
            log_message(f"Sample size: {baseline.sample_size}", 'info')
            
            print("✅ Baseline Calculation Complete!")
            print(f"\n📊 Baseline Statistics:")
            print(f"   Mean: {baseline.mean:.3f}")
            print(f"   Std: {baseline.std:.3f}")
            print(f"   Sample Size: {baseline.sample_size}")
            print(f"   Median: {baseline.median:.3f}")
            print(f"   Min: {baseline.min:.3f}, Max: {baseline.max:.3f}")
            
            status_display.value = f"✅ Baseline: μ={baseline.mean:.3f}, σ={baseline.std:.3f}"
            results_summary_display.value = f"Mean: {baseline.mean:.3f}<br>Std: {baseline.std:.3f}<br>Sample Size: {baseline.sample_size}"
            metrics_display.value = f"Mean: {baseline.mean:.3f}<br>Std: {baseline.std:.3f}<br>Range: {baseline.max - baseline.min:.3f}"
            
        except Exception as e:
            log_message(f"Baseline calculation failed: {str(e)}", 'error')
            print(f"❌ Baseline calculation failed: {e}")
            status_display.value = f"❌ Error: {e}"
    else:
        log_message("Running baseline calculation in demo mode...", 'warning')
        mean = np.mean(data)
        std = np.std(data)
        print("✅ Baseline Calculation Complete! (Demo Mode)")
        print(f"\n📊 Baseline Statistics (Demo):")
        print(f"   Mean: {mean:.3f}")
        print(f"   Std: {std:.3f}")
        update_status("Baseline calculation complete (demo)", 80)
        status_display.value = f"✅ Baseline complete (demo): μ={mean:.3f}, σ={std:.3f}"

def execute_comprehensive_workflow():
    """Execute comprehensive SPC workflow."""
    log_message("Comprehensive SPC Workflow", 'info')
    log_message("Running all SPC steps in sequence...", 'info')
    
    print("🚀 Comprehensive SPC Workflow")
    print("=" * 60)
    print("\nRunning all SPC steps...\n")
    
    # Step 1: Baseline
    update_status("Step 1/5: Baseline Calculation", 10)
    log_message("=" * 60, 'info')
    log_message("Step 1: Baseline Calculation", 'info')
    print("Step 1: Baseline Calculation")
    print("-" * 60)
    execute_baseline_calculation()
    print()
    
    # Step 2: Control Charts
    update_status("Step 2/5: Control Charts", 30)
    log_message("=" * 60, 'info')
    log_message("Step 2: Control Charts", 'info')
    print("Step 2: Control Charts")
    print("-" * 60)
    execute_control_charts()
    print()
    
    # Step 3: Process Capability
    update_status("Step 3/5: Process Capability", 50)
    log_message("=" * 60, 'info')
    log_message("Step 3: Process Capability", 'info')
    print("Step 3: Process Capability")
    print("-" * 60)
    execute_process_capability()
    print()
    
    # Step 4: Control Rules
    update_status("Step 4/5: Control Rules", 70)
    log_message("=" * 60, 'info')
    log_message("Step 4: Control Rules", 'info')
    print("Step 4: Control Rules")
    print("-" * 60)
    execute_control_rules()
    print()
    
    # Step 5: Multivariate SPC
    update_status("Step 5/5: Multivariate SPC", 90)
    log_message("=" * 60, 'info')
    log_message("Step 5: Multivariate SPC", 'info')
    print("Step 5: Multivariate SPC")
    print("-" * 60)
    execute_multivariate_spc()
    print()
    
    log_message("=" * 60, 'info')
    log_message("✅ Comprehensive SPC Workflow Complete!", 'success')
    print("=" * 60)
    print("✅ Comprehensive SPC Workflow Complete!")
    print("=" * 60)
    
    update_status("Comprehensive workflow complete", 100)
    status_display.value = "✅ Comprehensive workflow complete"
    results_summary_display.value = "All SPC steps completed successfully"

# Wire up execute button
execute_button.on_click(execute_spc_analysis)

def export_report(b):
    """Export SPC report."""
    if not spc_reports.get('latest'):
        warning_display.value = "<span style='color: orange;'>⚠️ No SPC report available. Run SPC analysis first.</span>"
        log_message("Export attempted but no report available", 'warning')
        return
    
    try:
        report = spc_reports['latest']
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"spc_report_{timestamp}.txt"
        log_message(f"Report ready for export: {filename}", 'info')
        warning_display.value = f"<span style='color: green;'>✅ Report ready: {filename}</span>"
    except Exception as e:
        log_message(f"Export failed: {str(e)}", 'error')
        warning_display.value = f"<span style='color: red;'>❌ Export failed: {str(e)}</span>"

export_button.on_click(export_report)

print("✅ Execution functions initialized!")

✅ Execution functions initialized!


In [4]:
# ============================================
# Complete Interface Layout and Display
# ============================================

# Update view based on SPC type
def update_view(change):
    """Update visualization mode based on SPC type."""
    spc_type_val = change['new']
    if spc_type_val == 'control_charts':
        viz_mode.value = 'charts'
    elif spc_type_val == 'capability':
        viz_mode.value = 'capability'
    elif spc_type_val == 'multivariate':
        viz_mode.value = 'multivariate'
    elif spc_type_val == 'rules':
        viz_mode.value = 'rules'
    elif spc_type_val == 'baseline':
        viz_mode.value = 'baseline'
    elif spc_type_val == 'comprehensive':
        viz_mode.value = 'report'

spc_type.observe(update_view, names='value')

# Complete interface layout
main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel], layout=Layout(width='100%', border='2px solid #333', padding='10px')),
    bottom_panel
])

display(main_layout)

## Summary and Next Steps

### Key Takeaways

You've learned how to:

1. **Generate Control Charts**: Create X-bar, R, S, Individual, and Moving Range charts with real-time progress tracking
2. **Analyze Process Capability**: Calculate Cp, Cpk, Pp, Ppk indices and rate process capability
3. **Perform Multivariate SPC**: Use Hotelling T² and PCA-based methods for multivariate monitoring
4. **Detect Control Rule Violations**: Apply Western Electric and Nelson rules to identify process issues
5. **Establish Baselines**: Calculate baseline statistics and update with adaptive limits
6. **Monitor SPC Progress**: Use the status bar and logs section to track SPC operations in real-time
7. **Interpret SPC Results**: Understand control charts, capability indices, and rule violations to make process improvement decisions

### Interface Features

The notebook provides a comprehensive SPC interface with:

- **Status Progress Bar**: Visual indication of SPC analysis progress (0-100%)
- **Real-Time Status Display**: Shows current operation, progress percentage, and elapsed time
- **Detailed Logs Section**: Timestamped execution logs with emoji indicators:
  - ℹ️ Information messages
  - ✅ Success messages
  - ⚠️ Warning messages
  - ❌ Error messages (with full tracebacks)
- **Time Tracking**: Automatic tracking of execution time for all SPC operations
- **Error Handling**: Comprehensive error messages displayed in both the logs and status sections

### Best Practices

- **Establish Baselines First**: Always establish a baseline before monitoring with control charts
- **Use Appropriate Chart Types**: Choose the right chart type based on your data (subgrouped vs. individual)
- **Monitor Capability Regularly**: Track process capability indices (Cpk, Ppk) to ensure process meets specifications
- **Review Rule Violations**: Investigate all control rule violations to identify process issues early
- **Set Appropriate Limits**: Use 3-sigma limits for control and 2-sigma for warnings
- **Monitor Logs**: Check the logs section for detailed execution information and any warnings
- **Review Progress**: Use the status bar to monitor long-running SPC operations

### Next Steps

- Explore the SPC module API for advanced use cases
- Integrate SPC analysis into your quality assessment workflows
- Customize control limits and rules for your specific manufacturing processes
- Review SPC logs to optimize performance and identify process issues
- Export SPC reports for documentation and sharing
- Integrate SPC with quality assessment for comprehensive quality monitoring

### Additional Resources

- SPC Module Documentation: `src/am_qadf/analytics/spc/`
- Test Examples: `tests/unit/analytics/spc/` and `tests/integration/spc/`
- SPC Implementation Plan: `implementation_plans/SPC_MODULE_IMPLEMENTATION.md`

---

**Congratulations!** You've completed the Statistical Process Control notebook. You now have the tools to monitor and control manufacturing processes, analyze process capability, detect process issues, and establish process baselines. The real-time progress tracking and detailed logging features help you monitor and troubleshoot SPC operations effectively. 🎉